In [9]:
month_name = {1:"Januari", 2:"Februari", 3:"Mars", 4:"April", 5:"Maj", 6:"Juni", 7:"Juli", 8:"Augusti", 9:"September", 10:"Oktober", 11:"November", 12:"December"}

import os, sys 
import geopandas as gpd 
import pandas as pd
import numpy as np 

from shapely.geometry import LineString, Polygon, Point

import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import make_axes_locatable

import folium 
from folium.plugins import MarkerCluster, HeatMap, FeatureGroupSubGroup
from branca.element import Template, MacroElement

sys.path.append("../bin/")
from UtilityFunctions import * 

with open("template.txt", "r") as fid:
    template = fid.read()
    
gdf_cykel = gpd.read_file("../../PythonSpatial/_local/maps/_local/södermalm_shp/gdf_cykel.shp").to_crs("EPSG:4326")

def format_popup(row):
    row2 = row.replace(to_replace = [None], value="Ingen information")
    string = "Datum:" + month_name[int(row2["Månad"])] + " " + "{:.0f}".format(row2["År"]) +  "<br>" + \
        "Olycksväg: " + row2["Olycksväg_"] + "<br>" + \
        "Vägomständ: " + row2["Vägomständ"] + "<br>" +\
        "Väglag: " + row2["Väglag"] + "<br>" + \
        "Ljusförhållanden: " + row2["Ljusförhål"] + "<br>" + \
        "Platstyp: " + row2["Platstyp"] 
    return string

icons = {
    'G1 (cykel singel)': {"icon":'ok-sign', "color":"orange"},
    'C (cykel/moped-motorfordon)': {"icon": 'ok-sign', "color":"orange"},
    'Cc (cykel-motorfordon)': {"icon": 'ok-sign', "color":"darkpurple"},
    'G4 (cykel-cykel)': {"icon": 'ok-sign', "color":"darkgreen"},
    'G5 (cykel-moped)': {"icon": 'ok-sign', "color":"darkgreen"},
}

sev_colors = {
    'Ej personskadeolycka': "#34495E",
    'Lindrig olycka (ISS 1-3)': "#A569BD",
    'Måttlig olycka (ISS 4-8)': "#F1C40F",
    'Allvarlig olycka (ISS 9-)': "#A93226",
    'Dödsolycka':"#000000",
    'Dödsolycka, ej officiell statistik':"#000000",
    'Olycka med osäkra skador': "#999999",
    }

settings = lambda name, severity: dict(
    weight = 1, 
    fill = True, 
    fill_opacity = 0.9, 
    color = "black", 
    tooltip = name,
    fill_color = sev_colors[severity], 
    radius = 10,
    
)

xmid,ymid = get_middle(gdf_cykel.geometry.total_bounds)
m = folium.Map(location = (ymid, xmid), zoom_start = 14, control_scale = True)

for year, gdf_sub in gdf_cykel.groupby("År"):
    
    fg_year = folium.FeatureGroup(name = year, show=False)
    
#     fg_heat = folium.FeatureGroup(name = "{:.0f}".format(year) + "--heat", show = False)
#     HeatMap(list(zip(gdf_sub.geometry.y, gdf_sub.geometry.x))).add_to(fg_heat)
    
    for name, frame in gdf_sub.groupby("MAIN_ATTR0"):
        for severity, sev_frame in frame.groupby("Sammanvägd"):
            for index, row in sev_frame.iterrows():
                fg_year.add_child(
                    folium.CircleMarker(
                        location = (row.geometry.y, row.geometry.x),
                        popup = folium.Popup(format_popup(row), min_width = 300, max_width=300), 
                        **settings(name, severity)
                    )
                )
    
    fg_year.add_to(m)
    
#     fg_heat.add_to(m)
    
folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)


folium.LayerControl(collapsed=False).add_to(m)

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

m.save("../index.html")
print("----------- DONE! -----------")

----------- DONE! -----------
